In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from albumentations import CenterCrop, RandomRotate90, GridDistortion, HorizontalFlip, VerticalFlip, RandomBrightnessContrast
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Bidirectional, Activation, MaxPool2D, Conv2DTranspose, LayerNormalization, Concatenate, Input, ZeroPadding2D,ConvLSTM2D,LSTM,GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate,TimeDistributed, Add, add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.applications import InceptionResNetV2, DenseNet121, DenseNet201, MobileNetV2,ResNet50,VGG16,VGG19, NASNetLarge,EfficientNetB7,EfficientNetV2S, EfficientNetV2M,EfficientNetV2L
from tensorflow.keras.metrics import Recall, Precision, Accuracy, MeanIoU
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

In [ ]:
### FLoMo-NetB2 ###

import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, ReLU, Add, LeakyReLU, Multiply, Dense, GlobalAveragePooling2D, 
    GlobalMaxPooling2D, Lambda,Activation, Concatenate, UpSampling2D, multiply, DepthwiseConv2D, SeparableConv2D, LayerNormalization
)


def discrete_cosine_transform(inputs, num_freq_components=16):
    """
    Computes 2D DCT (spatial frequency transform) over H and W axes.

    Args:
        inputs: Tensor of shape [B, H, W, C]
        num_freq_components: How many top-left frequencies to keep

    Returns:
        Tensor of shape [B, num_freq_components, num_freq_components, C]
    """
    # Permute to [B, C, H, W]
    x = tf.transpose(inputs, perm=[0, 3, 1, 2])
    B, C, H, W = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3]

    # Flatten across W and apply DCT
    x = tf.reshape(x, [-1, W])
    x = tf.signal.dct(x, type=2, norm='ortho')  # DCT over W

    # Restore shape and DCT over H
    x = tf.reshape(x, [B * C, H, W])
    x = tf.transpose(x, perm=[0, 2, 1])  # swap H <-> W
    x = tf.signal.dct(x, type=2, norm='ortho')  # DCT over H

    # Restore final shape and permute to [B, H, W, C]
    x = tf.reshape(x, [B, C, W, H])
    x = tf.transpose(x, perm=[0, 2, 3, 1])

    # Truncate to low frequencies
    return x[:, :num_freq_components, :num_freq_components, :]


def gated_channel_attention_with_dct(inputs, initial_reduction_ratio=8, num_freq_components=16):
    channels = inputs.shape[-1]

    dct_features = discrete_cosine_transform(inputs, num_freq_components)

    avg_pool_orig = layers.GlobalAveragePooling2D()(inputs)
    avg_pool_dct = layers.GlobalAveragePooling2D()(dct_features)

    max_pool_orig = layers.GlobalMaxPooling2D()(inputs)
    max_pool_dct = layers.GlobalMaxPooling2D()(dct_features)

    intermediate_pool = tf.nn.avg_pool2d(inputs, ksize=4, strides=2, padding="SAME")
    intermediate_pool = tf.nn.avg_pool2d(intermediate_pool, ksize=4, strides=2, padding="SAME")
    intermediate_pool = layers.GlobalAveragePooling2D()(intermediate_pool)

    pooling_features = layers.Concatenate()([avg_pool_orig, max_pool_orig, avg_pool_dct, max_pool_dct, intermediate_pool])
    normalized_features = layers.LayerNormalization()(pooling_features)

    dynamic_reduction_ratio = tf.Variable(initial_reduction_ratio, trainable=True, dtype=tf.float32, name="dynamic_reduction_ratio")
    reduced_channels = tf.cast(channels // tf.math.maximum(dynamic_reduction_ratio, 1), dtype=tf.int32)  

    bottleneck = layers.Dense(reduced_channels, activation="relu")(normalized_features)
    expanded = layers.Dense(channels, activation="sigmoid")(bottleneck)

    gate = tf.expand_dims(tf.expand_dims(expanded, axis=1), axis=2)

    output = layers.Multiply()([inputs, gate])

    return output

def adaptive_weighted_spatial_attention(inputs, kernel_size=7):
    avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)

    adaptive_weights = tf.Variable([0.5, 0.5], trainable=True, dtype=tf.float32, name="adaptive_weights")
    adaptive_weights = tf.nn.softmax(adaptive_weights) 

    adaptive_pool = adaptive_weights[0] * avg_pool + adaptive_weights[1] * max_pool

    depthwise_attention = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(adaptive_pool)
    pointwise_attention = layers.Conv2D(1, kernel_size=1, activation="sigmoid")(depthwise_attention)

    output = layers.Multiply()([inputs, pointwise_attention])

    return output

### Dual-Attention Selective Aggregator (DASA)
def DASA(inputs, reduction_ratio=8, kernel_size=7, kernel_sizes=[3, 5], patch_size=4, name="hacam_ws"):
    """
    Dual-Attention Selective Aggregator (DASA)
    """
    channel_attention = gated_channel_attention_with_dct(inputs, initial_reduction_ratio=reduction_ratio)

    combined_attention_output = adaptive_weighted_spatial_attention(channel_attention, kernel_size=kernel_size)

    return combined_attention_output

def squeeze_and_excitation(input_tensor, filters, name):
    se_gap = GlobalAveragePooling2D(name=f"{name}_se_gap")(input_tensor)

    se_fc1 = Dense(filters // 4, activation="relu", name=f"{name}_se_fc1")(se_gap)
    se_fc2 = Dense(filters, activation="sigmoid", name=f"{name}_se_fc2")(se_fc1)

    se_weights = tf.expand_dims(tf.expand_dims(se_fc2, axis=1), axis=2, name=f"{name}_se_expand")
    se_output = Multiply(name=f"{name}_se_scale")([input_tensor, se_weights])
    
    return se_output


### Local-Global Mixture of Experts (LoGo-MoE)
def LoGoMoE(input_tensor, num_filters, stage_index, name_prefix="",
               downsample=False, routing_mode="soft", top_k=2,
               add_entropy_loss=False, lambda_entropy=0.01):
    uid = tf.keras.backend.get_uid('LoGoSwitch')

    # ──────────────────────────────────────
    # Expert Branches (Local Multi-Scale)
    # ──────────────────────────────────────
    def expert_branch(x, k, d, name):
        x = DepthwiseConv2D(kernel_size=k, dilation_rate=d, padding="same", name=f"{name}_dw")(x)
        x = Conv2D(num_filters, 1, padding="same", activation="relu", name=f"{name}_pw")(x)
        return x

    expert_cfgs = [(3, 1), (3, 2), (3, 3), (5, 1), (5, 2), (5, 3), (7, 1)]
    expert_outputs = [expert_branch(input_tensor, k, d, f"{name_prefix}_exp_k{k}_d{d}_{uid}") for k, d in expert_cfgs]
    stacked_experts = tf.stack(expert_outputs, axis=1)  # [B, N, H, W, C]

    # ──────────────────────────────────────
    # Routing Scores
    # ──────────────────────────────────────
    pooled = GlobalAveragePooling2D()(input_tensor)
    routing_logits = Dense(len(expert_outputs), activation=None, name=f"{name_prefix}_router_logits_{uid}")(pooled)

    if routing_mode == "topk":
        values, indices = tf.math.top_k(routing_logits, k=top_k, sorted=False)
        mask = tf.reduce_sum(tf.one_hot(indices, depth=len(expert_outputs)), axis=1)
        masked_logits = routing_logits - 1e9 * (1.0 - mask)
        routing_scores = tf.nn.softmax(masked_logits, axis=-1)
    else:
        routing_scores = tf.nn.softmax(routing_logits, axis=-1)

    routing_scores_exp = tf.expand_dims(tf.expand_dims(tf.expand_dims(routing_scores, -1), -1), -1)
    weighted_experts = stacked_experts * routing_scores_exp
    fused_local = tf.reduce_sum(weighted_experts, axis=1, name=f"{name_prefix}_local_fused_{uid}")

    # ──────────────────────────────────────
    # Optional Entropy Loss
    # ──────────────────────────────────────
    if add_entropy_loss:
        entropy = -tf.reduce_sum(routing_scores * tf.math.log(routing_scores + 1e-8), axis=-1)
        mean_entropy = tf.reduce_mean(entropy)
        entropy_loss = lambda_entropy * mean_entropy
    else:
        entropy_loss = None

    # ──────────────────────────────────────
    # Global Context Branch
    # ──────────────────────────────────────
    global_pool = GlobalAveragePooling2D(keepdims=True)(input_tensor)  # [B, 1, 1, C]
    global_feat = Conv2D(num_filters, 1, activation="relu", name=f"{name_prefix}_global_{uid}")(global_pool)

    global_gate = Dense(num_filters // 4, activation="relu")(global_feat)
    global_gate = Dense(num_filters, activation="sigmoid")(global_gate)
    global_gate = tf.expand_dims(tf.expand_dims(global_gate, 1), 1)

    h = tf.shape(input_tensor)[1]
    w = tf.shape(input_tensor)[2]
    global_feat = tf.tile(global_feat, [1, h, w, 1])  # Safe dynamic tiling

    # ──────────────────────────────────────
    # Adaptive Fusion
    # ──────────────────────────────────────
    diff = tf.keras.layers.Subtract(name=f"{name_prefix}_diff_{uid}")([fused_local, global_feat])
    gate = Conv2D(num_filters, 3, padding="same", name=f"{name_prefix}_gateconv_{uid}")(diff)
    gate = BatchNormalization(name=f"{name_prefix}_gate_bn_{uid}")(gate)
    gate = Activation("sigmoid", name=f"{name_prefix}_gate_sigmoid_{uid}")(gate)

    fused = Add(name=f"{name_prefix}_switch_{uid}")([
        Multiply()([gate, fused_local]),
        Multiply()([1.0 - gate, global_feat])
    ])

    # ──────────────────────────────────────
    # Refinement
    # ──────────────────────────────────────
    fused = DepthwiseConv2D(3, padding="same", activation="gelu", name=f"{name_prefix}_refine_dw_{uid}")(fused)
    fused = BatchNormalization(name=f"{name_prefix}_refine_bn_{uid}")(fused)

    if downsample:
        fused = Conv2D(num_filters, 3, strides=2, padding="same", activation="gelu",
                       name=f"{name_prefix}_down_{stage_index}")(fused)

    return fused, entropy_loss


##### LoGoMoE Encoder
def LoGoMoEEnc(inputs, num_filters_initial=64, reduction_ratio=8, kernel_size=7, kernel_sizes=[3, 5, 7],
               routing_mode="soft", top_k=2, name="logo_encoder"):

    x = inputs
    num_filters = num_filters_initial
    encoder_features = []

    for stage_index in range(1, 5):
        stage_name = f"{name}_stage_{stage_index}"

        x, _ = LoGoMoE(
            x,
            num_filters=num_filters,
            stage_index=stage_index,
            name_prefix=stage_name,
            routing_mode=routing_mode,
            top_k=top_k,
            downsample=True,
            add_entropy_loss=False
        )

        if stage_index >= 3:
            x = DASA(
                x,
                reduction_ratio=reduction_ratio,
                kernel_size=kernel_size,
                kernel_sizes=kernel_sizes,
                name=f"{stage_name}_hacam"
            )

        encoder_features.append(x)
        num_filters *= 2

    return encoder_features



### False Positive/Negative Corrective Attention Module (FPFN-CAM)
class FPFNCAM(tf.keras.layers.Layer):
    def __init__(self, filters, name="FPFNCAM", **kwargs):
        super(FPFNCAM, self).__init__(name=name, **kwargs)
        self.filters = filters

        # Projection
        self.enc_proj = Conv2D(filters, 1, activation="relu", name=f"{name}_enc_proj")
        self.dec_proj = Conv2D(filters, 1, activation="relu", name=f"{name}_dec_proj")

        # Cross interaction
        self.cross_gate = Conv2D(filters, 1, activation="sigmoid", name=f"{name}_cross_gate")
        self.fused_conv = Conv2D(filters, 3, padding="same", activation="relu", name=f"{name}_fused_conv")

        # Logits-like entropy estimation
        self.depthwise_entropy = DepthwiseConv2D(3, padding="same", activation="relu", name=f"{name}_entropy_depth")
        self.logits_like = Conv2D(filters, 1, padding="same", name=f"{name}_logits")

        # Normalization
        self.entropy_norm = LayerNormalization(name=f"{name}_entropy_norm")
        self.cosine_norm = LayerNormalization(name=f"{name}_cosine_norm")

        # Fusion & FP/FN gates
        self.uncert_weight = Conv2D(1, 1, activation="sigmoid", name=f"{name}_uncert_weight")
        self.fpfn_gate = Conv2D(1, 3, padding="same", activation="sigmoid", name=f"{name}_fpfn_gate")
        self.fpfn_correction = Conv2D(filters, 3, padding="same", activation="relu", name=f"{name}_fpfn_correction")
        self.final_conv = Conv2D(filters, 1, activation="gelu", name=f"{name}_final")

    def call(self, enc_feat, dec_feat):
        # Feature projection
        enc_proj = self.enc_proj(enc_feat)
        dec_proj = self.dec_proj(dec_feat)

        # Resize encoder if needed
        if enc_proj.shape[1] > dec_proj.shape[1]:
            enc_proj = Conv2D(self.filters, 3, strides=2, padding="same", activation="relu")(enc_proj)
        elif enc_proj.shape[1] < dec_proj.shape[1]:
            enc_proj = UpSampling2D(size=2, interpolation="bilinear")(enc_proj)

        # Cross interaction
        cross = Multiply()([dec_proj, self.cross_gate(enc_proj)])
        fused = Add()([enc_proj, cross])
        fused = self.fused_conv(fused)

        # Entropy estimation
        x = self.depthwise_entropy(fused)
        x = self.logits_like(x)
        soft = Activation("softmax")(x)
        entropy = -tf.reduce_sum(soft * tf.math.log(soft + 1e-8), axis=-1, keepdims=True)
        entropy = self.entropy_norm(entropy)

        # Cosine difference
        norm_e = tf.nn.l2_normalize(enc_proj, axis=-1)
        norm_d = tf.nn.l2_normalize(dec_proj, axis=-1)
        cosine_diff = 1.0 - tf.reduce_sum(norm_e * norm_d, axis=-1, keepdims=True)
        cosine_diff = self.cosine_norm(cosine_diff)

        # Fuse uncertainty maps
        fused_uncert = Concatenate()([entropy, cosine_diff])
        fusion_weight = self.uncert_weight(fused_uncert)

        # FP/FN correction
        fpfn_mask = self.fpfn_gate(fused_uncert)
        correction = self.fpfn_correction(fused * fpfn_mask)

        # Modulate and correct
        modulated = fused * (1.0 - fusion_weight)
        corrected = Add()([modulated, correction, dec_proj])
        output = self.final_conv(corrected)

        return output

### ##### LoGoMoE-FPFNCAM Decoder
def LoGoMoEFPFNCAM(
    low_level, mid_level, high_level, deeper, bridge_features,
    filters_per_stage=[512, 256, 128, 64],
    reduction_ratio=8, kernel_size=7, kernel_sizes=[3, 5, 7],
    routing_mode="soft", top_k=2,  
    name="decoder"
):
    FPFNCAM_stage1 = FPFNCAM(filters=filters_per_stage[0], name=f"{name}_stage1_gcacm")
    FPFNCAM_stage2 = FPFNCAM(filters=filters_per_stage[1], name=f"{name}_stage2_gcacm")
    FPFNCAM_stage3 = FPFNCAM(filters=filters_per_stage[2], name=f"{name}_stage3_gcacm")
    FPFNCAM_stage4 = FPFNCAM(filters=filters_per_stage[3], name=f"{name}_stage4_gcacm")

    d1 = UpSampling2D(size=2, name=f"{name}_stage1_upsample")(bridge_features)
    d1, _ = LoGoMoE(d1, num_filters=filters_per_stage[0], stage_index=1, name_prefix=f"{name}_stage1",
                    routing_mode=routing_mode, top_k=top_k, downsample=False)
    d1 = DASA(d1, reduction_ratio=reduction_ratio, kernel_size=kernel_size,
            kernel_sizes=kernel_sizes, name=f"{name}_stage1_DASA")
    d1 = FPFNCAM_stage1(deeper, d1)

    d2 = UpSampling2D(size=2, name=f"{name}_stage2_upsample")(d1)
    d2, _ = LoGoMoE(d2, num_filters=filters_per_stage[1], stage_index=2, name_prefix=f"{name}_stage2",
                    routing_mode=routing_mode, top_k=top_k, downsample=False)
    d2 = DASA(d2, reduction_ratio=reduction_ratio, kernel_size=kernel_size,
            kernel_sizes=kernel_sizes, name=f"{name}_stage2_DASA")
    d2 = FPFNCAM_stage2(high_level, d2)

    d3 = UpSampling2D(size=2, name=f"{name}_stage3_upsample")(d2)
    d3, _ = LoGoMoE(d3, num_filters=filters_per_stage[2], stage_index=3, name_prefix=f"{name}_stage3",
                    routing_mode=routing_mode, top_k=top_k, downsample=False)
    d3 = DASA(d3, reduction_ratio=reduction_ratio, kernel_size=kernel_size,
            kernel_sizes=kernel_sizes, name=f"{name}_stage3_DASA")
    d3 = FPFNCAM_stage3(mid_level, d3)

    d4 = UpSampling2D(size=2, name=f"{name}_stage4_upsample")(d3)
    d4, _ = LoGoMoE(d4, num_filters=filters_per_stage[3], stage_index=4, name_prefix=f"{name}_stage4",
                    routing_mode=routing_mode, top_k=top_k, downsample=False)
    d4 = DASA(d4, reduction_ratio=reduction_ratio, kernel_size=kernel_size,
            kernel_sizes=kernel_sizes, name=f"{name}_stage4_DASA")
    d4 = FPFNCAM_stage4(low_level, d4)
    return d4


### Frequency-Aware Multi-Scale Refinement (FAMSR)
def FAMSR(input_tensor, filters, name="famsr"):
    # ───────────────────────────────────────────────
    # Frequency Branch (DCT → Low + High Frequency)
    # ───────────────────────────────────────────────
    dct_features = layers.Lambda(discrete_cosine_transform, name=f"{name}_dct")(input_tensor)

    low_freq = DepthwiseConv2D(kernel_size=3, padding="same", activation="gelu", name=f"{name}_low_freq_dw")(dct_features)
    low_freq = BatchNormalization(name=f"{name}_low_freq_bn")(low_freq)
    low_freq = Conv2D(filters, kernel_size=1, padding="same", activation="gelu", name=f"{name}_low_freq_pw")(low_freq)

    high_freq = tf.abs(dct_features - tf.reduce_mean(dct_features, axis=[1, 2], keepdims=True))
    high_freq = DepthwiseConv2D(kernel_size=3, padding="same", activation="gelu", name=f"{name}_high_freq_dw")(high_freq)
    high_freq = BatchNormalization(name=f"{name}_high_freq_bn")(high_freq)
    high_freq = Conv2D(filters, kernel_size=1, padding="same", activation="gelu", name=f"{name}_high_freq_pw")(high_freq)

    freq_refined = Add(name=f"{name}_freq_combined")([low_freq, high_freq])

    # ───────────────────────────────────────────────
    # Multi-scale Dilation Branch (Adaptive Fusion)
    # ───────────────────────────────────────────────
    adaptive_weights = GlobalAveragePooling2D(name=f"{name}_adaptive_weights_gap")(input_tensor)
    adaptive_weights = Dense(3, activation="softmax", name=f"{name}_adaptive_weights")(adaptive_weights)
    adaptive_weights = tf.expand_dims(tf.expand_dims(adaptive_weights, axis=1), axis=2)

    d4 = DepthwiseConv2D(kernel_size=3, dilation_rate=4, padding="same", activation="relu", name=f"{name}_d4")(input_tensor)
    d8 = DepthwiseConv2D(kernel_size=3, dilation_rate=8, padding="same", activation="relu", name=f"{name}_d8")(input_tensor)
    d12 = DepthwiseConv2D(kernel_size=3, dilation_rate=12, padding="same", activation="relu", name=f"{name}_d12")(input_tensor)

    multiscale_fused = Add(name=f"{name}_multi_scale_fused")([
        Multiply()([d4, adaptive_weights[..., 0:1]]),
        Multiply()([d8, adaptive_weights[..., 1:2]]),
        Multiply()([d12, adaptive_weights[..., 2:3]])
    ])
    
    multiscale_fused = Conv2D(filters, kernel_size=1, padding="same", activation="gelu", name=f"{name}_multi_scale_pw")(multiscale_fused)

    # ───────────────────────────────────────────────
    # Channel Attention (Squeeze-and-Excitation)
    # ───────────────────────────────────────────────
    se_gap = GlobalAveragePooling2D(name=f"{name}_se_gap")(multiscale_fused)
    se_fc1 = Dense(filters // 4, activation="relu", name=f"{name}_se_fc1")(se_gap)
    se_fc2 = Dense(filters, activation="sigmoid", name=f"{name}_se_fc2")(se_fc1)
    se_weights = tf.expand_dims(tf.expand_dims(se_fc2, axis=1), axis=2, name=f"{name}_se_expand")
    se_refined = Multiply(name=f"{name}_se_scaled")([multiscale_fused, se_weights])

    # ───────────────────────────────────────────────
    # Dynamic Retention (Gated Spatial Refinement)
    # ───────────────────────────────────────────────
    dynamic = DepthwiseConv2D(kernel_size=3, padding="same", activation="gelu", name=f"{name}_spatial_dw")(se_refined)
    dynamic = BatchNormalization(name=f"{name}_spatial_bn")(dynamic)
    dynamic = Conv2D(filters, kernel_size=1, padding="same", activation="gelu", name=f"{name}_spatial_pw")(dynamic)

    alpha = Dense(filters, activation="sigmoid", name=f"{name}_alpha")(GlobalAveragePooling2D()(dynamic))
    alpha = tf.expand_dims(tf.expand_dims(alpha, axis=1), axis=2)
    retained = Multiply(name=f"{name}_retained")([dynamic, alpha])

    # ───────────────────────────────────────────────
    # Final Spatial Attention Gating
    # ───────────────────────────────────────────────
    att_gap = GlobalAveragePooling2D(name=f"{name}_att_gap")(retained)
    att_gmp = GlobalMaxPooling2D(name=f"{name}_att_gmp")(retained)
    gate_signal = Concatenate(name=f"{name}_att_concat")([att_gap, att_gmp])
    gate_signal = Dense(filters // 4, activation="relu", name=f"{name}_att_fc1")(gate_signal)
    gate_signal = Dense(filters, activation="sigmoid", name=f"{name}_att_fc2")(gate_signal)
    gate_signal = tf.expand_dims(tf.expand_dims(gate_signal, axis=1), axis=2, name=f"{name}_att_expand")
    gated = Multiply(name=f"{name}_gated")([retained, gate_signal])

    # ───────────────────────────────────────────────
    # Final Combination with Frequency-Refined Path
    # ───────────────────────────────────────────────
    final = Add(name=f"{name}_final_combined")([gated, freq_refined])
    final = DepthwiseConv2D(kernel_size=3, padding="same", activation="gelu", name=f"{name}_final_dw")(final)
    final = BatchNormalization(name=f"{name}_final_bn")(final)
    final = Conv2D(filters, kernel_size=1, padding="same", activation="gelu", name=f"{name}_final_pw")(final)

    return final

def FLoMoNet(input_shape=(224, 224, 3), num_classes=4, routing_mode="soft", top_k=2):
    inputs = layers.Input(shape=input_shape, name="input_layer")
    s = tf.keras.layers.Lambda(lambda x: x / 255.0)(inputs)

    encoder_features = LoGoMoEEnc(
        s,
        num_filters_initial=64,
        reduction_ratio=8,
        kernel_size=7,
        kernel_sizes=[3, 5, 7],
        routing_mode=routing_mode,
        top_k=top_k,
        name="LoGoMoE_encoder"
    )
    e1, e2, e3, e4 = encoder_features

    bridge_features = FAMSR(e4, filters=1024, name="bridge_famsr")

    decoded_features = LoGoMoEFPFNCAM(
        e1, e2, e3, e4, bridge_features,
        filters_per_stage=[512, 256, 128, 64],
        reduction_ratio=8,
        kernel_size=7,
        kernel_sizes=[3, 5, 7],
        routing_mode=routing_mode,
        top_k=top_k,
        name="LoGoMoE_decoder"
    )

    outputs = layers.Conv2D(num_classes, kernel_size=1, activation="sigmoid", name="output_layer")(decoded_features)

    model = tf.keras.Model(inputs, outputs, name="FLoMo-Net")

    return model

# Build and summarize the model
model = FLoMoNet(input_shape=(224, 224, 3), num_classes=4, routing_mode="soft")
model.summary()

Model: "FLoMo-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_layer[0][0]']            
                                                                                                  
 global_average_pooling2d (Glob  (None, 3)           0           ['lambda[0][0]']                 
 alAveragePooling2D)                                                                              
                                                                                          